In [ ]:
import os
import time
import tweepy as tw
import pandas as pd
import numpy as np
import json
import pymongo
from pymongo import MongoClient


bearer_token = "AAAAAAAAAAAAAAAAAAAAAMlpVwEAAAAAyno6BFgWyrjtYUd19Omp9Jwp6E4%3DEJlrvFLXkuh9RNEb4BIZvU8s8KbP9YZlGnmUn3EE8ZGj0hdJRM"
consumer_key = "7SfGRroYyciJOmkMV0vud15o2"
consumer_secret = "yVofBL2bNkxPrRlUk0WFBP6bVzyan54riq5IeVGzCkVnQUjdqq"
access_token = "984375378-OsX5tC7GjFCnngMuWRFGxiTlbtDbjHz2eyI3reSB"
access_token_secret = "Tt0w1Qe3J50gRJU5UHJfnnRccBHOOKGpTnkBOZL7myCXv"

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)



clientcrypto = MongoClient('localhost', 27017)
db = clientcrypto.test
dbcrypto = clientcrypto.dataman
collection = dbcrypto["crypto"]



contatore = 0

response = collection.find({})
n = 0
list_nome = []
list_abbr = []
list_id = []

for document in response:
    list_nome.append(document['name'])
    list_abbr.append(document['ticker'])
    list_id.append(document['_id'])
    #with open("CSVcrypto", "a", encoding="utf-8") as F:
        #for i in range(0,len(list_nome)):
            
    
    #while i<= len(document['tweets']):
    #for element in document['tweets']:
        #txt.write(element['text'])
        #i = i+1
    #n=n+1
    #print("inseriti i valori numero", n)


# Funzioni

In [ ]:
def downloadTweets(query):
    tweets = client.search_recent_tweets(query, max_results=100)
    currentCRYPTOdataset = tweetsDataFrame(tweets)
    print("Presi i tweet di "+query)
    print(tweets.meta)
    for _ in range(3):
        if 'next_token' in tweets.meta:
            tweets = client.search_recent_tweets(query, max_results=100, next_token=tweets.meta['next_token'])
            df = tweetsDataFrame(tweets)
            currentCRYPTOdataset = currentCRYPTOdataset.append(df, ignore_index=True)
        else:
            break

        print("Presi i tweet di "+query)
    return currentCRYPTOdataset


def tweetsDataFrame(tweets):
    testo = list()
    id = list()
   
    for tweet in tweets.data:
        testo.append(tweet.text)
        id.append(tweet.id)
   
    data_vuoto = []
    df = pd.DataFrame(data_vuoto)

    df['id'] = id
    df['text'] = testo
   

    return df

# Caricamento Tweets

In [ ]:
n = 0    
for x in range(0,len(list_nomi)):
    nome = str(list_nomi[x])
    abbreviazione = str(list_abbre[x])
    ide = list_id[x]
    try:
        client = tw.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret,
                   access_token=access_token, access_token_secret=access_token_secret)
        df_nome = downloadTweets(nome)
        df_abbr = downloadTweets(abbreviazione)
        df_unione = df_nome.append(df_abbr)
        print('la lunghezza del dataframe di', nome, 'prima del drop è:', len(df_unione))
    #eliminazione duplicati 
        df_unione.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
        print('la lunghezza del dataframe di' , nome, 'dopo il drop è:', len(df_unione))
        df_load = df_unione.to_dict("records")   
        collection.update_many({"_id":ide}, {"$set":{"tweets":df_load}}) 
        print("ho inserio i documenti di", nome)
        contatore = contatore +1
    except Exception as e:
        print(e)
        if str(e) == "429 Too Many Requests":
                print("Pausa 15 minuti")
                time.sleep(15*60)
                client = tw.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret,
                   access_token=access_token, access_token_secret=access_token_secret)
                df_nome = downloadTweets(nome)
                df_abbr = downloadTweets(abbreviazione)
                df_unione = df_nome.append(df_abbr)
                print('la lunghezza del dataframe di', nome, 'prima del drop è:', len(df_unione))
                #eliminazione duplicati 
                df_unione.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
                print('la lunghezza del dataframe di' , nome, 'dopo il drop è:', len(df_unione))
                df_load = df_unione.to_dict("records")   
                collection.update_many({"_id":ide}, {"$set":{"tweets":df_load}}) 
                print("ho inserio i documenti di", nome)
                contatore=contatore+1
                
    print("Inserita crypto numero: ",contatore)
                
print("Finito")

CSV Backup

In [14]:
diz = {"id":list_id, "abbr":list_abbr, "nome": list_nome}
backup = pd.DataFrame(diz)
backup.to_csv(f'cryptolist.csv')

# Caricamento OHLC Data

In [ ]:
import pycoingecko
from datetime import datetime
from bson import ObjectId

In [ ]:
cg = pycoingecko.CoinGeckoAPI()
crypto_df = pd.read_csv('cryptolist.csv')
crypto_df = crypto_df.drop(columns='Unnamed: 0')

In [ ]:
id_list = cg.get_coins_list()
cg_list = pd.DataFrame(id_list)

In [ ]:
wormhole = cg_list.loc[cg_list['id'].str.contains('wormhole')].index
cg_list.drop(wormhole, inplace = True)
wrapped = cg_list.loc[cg_list['id'].str.contains('wrapped')].index
cg_list.drop(wrapped, inplace = True)

In [ ]:
crypto_df['abbr_low'] = np.nan

In [ ]:
for i in range(0,len(crypto_df)):
    crypto_df['abbr_low'][i] = crypto_df['abbr'][i].lower()

In [ ]:
coins_presenti = crypto_df.merge(cg_list, left_on=['abbr_low', 'nome'], right_on=['symbol', 'name'], how='inner')

In [ ]:
for i in range(0, len(coins_presenti)):
    idc = ObjectId(coins_presenti['id_x'][i])
    try:
        ohlc = cg.get_coin_ohlc_by_id(coins_presenti['id_y'][i], vs_currency='usd', days=7)
        print('Presi i dati di ', coins_presenti['abbr'][i])
        for j in range(0, len(ohlc)):
            ohlc[j][0] = datetime.fromtimestamp(ohlc[j][0]/1000).strftime(format = '%Y-%m-%d %X%z')
        ohlc_df = pd.DataFrame(ohlc)
        ohlc_df.columns = ['Data', 'Open', 'High', 'Low', 'Close']
        ohlc_dict = ohlc_df.to_dict("records")
        collection.update_many({"_id": idc}, {"$set": {"ohlc": ohlc_dict}})
    except Exception as e:
        print(e)
        if "429" in str(e):
            print("Pausa")
            time.sleep(3*60)
            ohlc = cg.get_coin_ohlc_by_id(coins_presenti['id_y'][i], vs_currency='usd', days=7)
            for x in range(0, len(ohlc)):
                ohlc[x][0] = datetime.fromtimestamp(ohlc[x][0]/1000).strftime(format = '%Y-%m-%d %X%z')
            ohlc_df = pd.DataFrame(ohlc)
            ohlc_df.columns = ['Data', 'Open', 'High', 'Low', 'Close']
            ohlc_dict = ohlc_df.to_dict("records")
            collection.update_many({"_id": idc}, {"$set": {"ohlc": ohlc_dict}})
            print('Caricati i dati di', coins_presenti['abbr'][i])
    
    print("Fine crypto ", i)